In [ ]:
import geopandas as gpd
import geocoder
import json
import pandas as pd
from shapely.geometry import LineString, Point, Polygon


In [ ]:
import pandas as pd
import requests
from io import StringIO

def get_api(resource_id, limit=1000, retries=3):
    offset = 0
    api_url = f"https://data.tycg.gov.tw/api/v1/rest/datastore/{resource_id}"


    all_data = pd.DataFrame()

    while True:
        try:
            params = {'limit': limit, 'offset': offset, 'format': 'csv'}
            response = requests.get(api_url, params=params)
            response.raise_for_status()  # 如果狀態碼不是 2xx，將拋出一個異常
            current_data = pd.read_csv(StringIO(response.text))
            # all_data = all_data.append(current_data, ignore_index=True)
            all_data = pd.concat([all_data, current_data], ignore_index=True)


            if len(current_data) < limit:
                break

            # 更新偏移量
            offset += limit
            print("OK")
        except requests.exceptions.RequestException as err:
            print("OOps: Something went wrong", err)

    return all_data

# get_api 函數獲取 API 數據
RID = 'f2219d56-1ca3-47c6-8dd8-ed9c352d0003'
api_data = get_api(RID)

if not api_data.empty:
    # DataFrame 保存為本地 CSV 文件
    OUTPUT_FILENAME = 'steal.csv'
    api_data.to_csv(OUTPUT_FILENAME, index=False)

    print(f"Data saved to {OUTPUT_FILENAME}")
else:
    print("Failed to retrieve API data.")


OK
OK
OK
Data saved to steal.csv


In [ ]:

print("原始資料:")
print(api_data)

new_order = ['_id','year', 'month', 'date', 'time', 'breau', 'station', 'lon', 'lat', 'type']
api_data = api_data[new_order]

df_loc = api_data[api_data['breau'] == '中壢分局']
print(df_loc)



原始資料:
      date breau  month  year station         lon   _id     time  type  \
0       24  龍潭分局     12   111     中興所  121.235578     1  1111224  汽車竊盜   
1       12  平鎮分局     12   111     龍岡所  121.248011     2  1111212  汽車竊盜   
2       30  龍潭分局     11   111     龍潭所  121.218214     3  1111130  汽車竊盜   
3       25  楊梅分局     11   111     草湳所  121.164441     4  1111125  汽車竊盜   
4       14  平鎮分局     11   111     平鎮所  121.218423     5  1111114  汽車竊盜   
...    ...   ...    ...   ...     ...         ...   ...      ...   ...   
3304     3  中壢分局     11   101     龍興所  121.243400  3305  1011103  汽車竊盜   
3305    17  平鎮分局      6   101     北勢所  121.212800  3306  1010617  汽車竊盜   
3306    29  平鎮分局      4   100     平鎮所  121.213090  3307  1000429  汽車竊盜   
3307     0  龜山分局      1   991     龜山所  121.346900  3308   991010  汽車竊盜   
3308     4  中壢分局     90   990     龍興所  121.257671  3309   990904  汽車竊盜   

            lat  
0     24.874379  
1     24.917239  
2     24.875023  
3     24.899386  
4     24.919823

In [ ]:
new_order = ['_id','year', 'month', 'date', 'time', 'breau', 'station', 'lon', 'lat', 'type']
newcolumns = {
    '_id': '序號',
    'year': '年',
    'month': '月',
    'date': '日',
    'time': '日期',
    'breau': '分局',
    'station': '所',
    'lon': '經度',
    'lat': '緯度',
    'type': '種類',
}

df_loc.drop('_id', axis=1, inplace=True)


df_loc.rename(columns=newcolumns, inplace=True)


# 將日期欄位轉換為西元年
df_loc['日期'] = (df_loc['年'] + 1911).astype(str) + df_loc['月'].astype(str).str.zfill(2) + df_loc['日'].astype(str).str.zfill(2)

# 將日期欄位等於 "29019004" 的整筆資料從 df_loc 中刪除 (怪怪的值)
df_loc = df_loc[df_loc['日期'] != '29019004']


print(df_loc)








        年   月   日        日期    分局    所          經度         緯度    種類
23    110  12  14  20211214  中壢分局  龍興所  121.258911  24.934926  汽車竊盜
27    110  12   1  20211201  中壢分局  文化所  121.249124  24.957730  汽車竊盜
30    110  11   9  20211109  中壢分局  普仁所  121.236280  24.963268  汽車竊盜
31    110  11   7  20211107  中壢分局  普仁所  121.231388  24.956273  汽車竊盜
34    110  10  31  20211031  中壢分局  興國所  121.218099  24.975003  汽車竊盜
...   ...  ..  ..       ...   ...  ...         ...        ...   ...
3294  104   1   3  20150103  中壢分局  龍興所  121.243300  24.928910  汽車竊盜
3297  104   1   1  20150101  中壢分局  仁愛所  121.231200  24.954940  汽車竊盜
3298  104   1   1  20150101  中壢分局  龍興所  121.237100  24.933020  汽車竊盜
3303  102   7  31  20130731  中壢分局  仁愛所  121.255900  24.946490  汽車竊盜
3304  101  11   3  20121103  中壢分局  龍興所  121.243400  24.940130  汽車竊盜

[681 rows x 9 columns]


<ipython-input-4-752128231ae2>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc.drop('_id', axis=1, inplace=True)
<ipython-input-4-752128231ae2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc.rename(columns=newcolumns, inplace=True)
<ipython-input-4-752128231ae2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc['日期'] = (df_loc['年'] + 1911).astype(str) + 

In [ ]:
# df_loc.to_csv('steal_ETL.csv', index=False, encoding='UTF-8')
df_loc.to_csv('steal_ETL.csv', index=False, encoding='UTF-8')